In [ ]:
import pandas as pd
import numpy as np

import os
from pathlib import Path

import plotly
import plotly.graph_objs as go

from tsfresh import extract_features, select_features
from tsfresh.feature_extraction.settings import MinimalFCParameters, from_columns, EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostClassifier

import torch
from torch import nn
from torch import optim

from tqdm import tqdm

In [2]:
# define relative path
path = Path.cwd()

# define seed
SEED = 100

In [3]:
# read the data
path_to_data = os.path.join(path, 'data', 'physionet2017.csv')
df = pd.read_csv(path_to_data, index_col=[-2])
df.head()


,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,label
name,,,,,,,,,,,,,,,,,,,,,
A00/A00001,0.035032,0.037155,0.044586,0.063694,0.076433,0.085987,0.089172,0.083864,0.072187,0.061571,...,0.020170,0.008493,0.000000,-0.007431,-0.012739,-0.015924,-0.019108,-0.023355,-0.022293,0
A00/A00002,-0.035288,-0.032573,-0.030945,-0.029859,-0.031488,-0.034202,-0.037459,-0.040717,-0.043974,-0.047231,...,-0.002714,-0.001629,-0.001086,-0.000543,-0.000543,0.000000,0.000000,0.000543,0.001086,0
A00/A00003,-0.303922,-0.261438,-0.222222,-0.192810,-0.176471,-0.163399,-0.147059,-0.130719,-0.117647,-0.107843,...,-0.339869,-0.346405,-0.339869,-0.323529,-0.297386,-0.264706,-0.209150,-0.117647,-0.065359,0
A00/A00004,0.109467,0.117604,0.128698,0.142012,0.153107,0.161982,0.170118,0.176036,0.181213,0.184911,...,0.846154,0.780325,0.640533,0.467456,0.298077,0.165680,0.085799,0.012574,0.013314,1
A00/A00005,-0.019856,-0.017148,-0.014440,-0.011733,-0.009928,-0.008123,-0.006318,-0.004513,-0.003610,-0.001805,...,-0.347473,-0.306859,-0.260830,-0.214801,-0.168773,-0.124549,-0.083935,-0.051444,0.000000,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8528 entries, A00/A00001 to A08/A08528
Columns: 2001 entries, 0 to label
dtypes: float64(2000), int64(1)
memory usage: 130.3+ MB


In [5]:
# labels info
info_dict = {0: 'Normal', 1: 'AF', 2: 'Other', 3: 'Noise'}

In [7]:
# # data visualization
# def visual(row):
#     name = row.name.replace('/', '_')
#     row = row.values
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(y=row[:-1]))
#     fig.update_layout(title={
#                             'text': info_dict[row[-1]],
#                             'font_size': 24,
#                             'y':0.9,
#                             'x':0.5})
#     fig.write_html(os.path.join('figures', f'{name}.html'))

# df.apply(visual, axis=1)

In [8]:
# labels visualization
for label in df['label'].unique():
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=df[df['label'] == label].iloc[1, :-1]))
    fig.update_layout(title=info_dict[label])
    fig.show()

In [4]:
# delete duplicates
df = df.drop_duplicates()

In [6]:
# check NaNs
df.isna().sum().sum()

0

In [7]:
# label variance
df['label'].value_counts()

0    5074
2    2415
1     758
3     279
Name: label, dtype: int64

In [5]:
# select features and target
X = df.drop(columns={'label'})
y = df['label']
X.head()

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
name,,,,,,,,,,,,,,,,,,,,,
A00/A00001,0.035032,0.037155,0.044586,0.063694,0.076433,0.085987,0.089172,0.083864,0.072187,0.061571,...,0.038217,0.020170,0.008493,0.000000,-0.007431,-0.012739,-0.015924,-0.019108,-0.023355,-0.022293
A00/A00002,-0.035288,-0.032573,-0.030945,-0.029859,-0.031488,-0.034202,-0.037459,-0.040717,-0.043974,-0.047231,...,-0.003800,-0.002714,-0.001629,-0.001086,-0.000543,-0.000543,0.000000,0.000000,0.000543,0.001086
A00/A00003,-0.303922,-0.261438,-0.222222,-0.192810,-0.176471,-0.163399,-0.147059,-0.130719,-0.117647,-0.107843,...,-0.330065,-0.339869,-0.346405,-0.339869,-0.323529,-0.297386,-0.264706,-0.209150,-0.117647,-0.065359
A00/A00004,0.109467,0.117604,0.128698,0.142012,0.153107,0.161982,0.170118,0.176036,0.181213,0.184911,...,0.844675,0.846154,0.780325,0.640533,0.467456,0.298077,0.165680,0.085799,0.012574,0.013314
A00/A00005,-0.019856,-0.017148,-0.014440,-0.011733,-0.009928,-0.008123,-0.006318,-0.004513,-0.003610,-0.001805,...,-0.381769,-0.347473,-0.306859,-0.260830,-0.214801,-0.168773,-0.124549,-0.083935,-0.051444,0.000000


In [6]:
# split to valid data
X, X_val, y, y_val = train_test_split(X, y, random_state=SEED, stratify=y, test_size=0.1, shuffle=True)
# X_val.to_csv(os.path.join('validation', 'data.csv'))
# y_val.to_csv(os.path.join('validation', 'target.csv'))

# Application of various technologies for classification

## 1. tsfresh + catboost

In [8]:
# vectorized data
data_long = pd.DataFrame({0: X.values.flatten(),
                          1: X.index.repeat(X.shape[1])})
data_long.head()

,0,1
0,0.088235,A04/A04495
1,0.100000,A04/A04495
2,0.117647,A04/A04495
3,0.135294,A04/A04495
4,0.147059,A04/A04495


In [21]:
# calculate features
extracted_features = extract_features(data_long, column_id=1, \
                                      impute_function=impute, default_fc_parameters=EfficientFCParameters()).sort_index()
# extracted_features.to_csv('EfficientFCParametrs.csv')
extracted_features.head()

Feature Extraction: 100%|██████████| 40/40 [51:52<00:00, 77.81s/it]   


,0__variance_larger_than_standard_deviation,0__has_duplicate_max,0__has_duplicate_min,0__has_duplicate,0__sum_values,0__abs_energy,0__mean_abs_change,0__mean_change,0__mean_second_derivative_central,0__median,...,0__permutation_entropy__dimension_6__tau_1,0__permutation_entropy__dimension_7__tau_1,0__query_similarity_count__query_None__threshold_0.0,"0__matrix_profile__feature_""min""__threshold_0.98","0__matrix_profile__feature_""max""__threshold_0.98","0__matrix_profile__feature_""mean""__threshold_0.98","0__matrix_profile__feature_""median""__threshold_0.98","0__matrix_profile__feature_""25""__threshold_0.98","0__matrix_profile__feature_""75""__threshold_0.98",0__mean_n_absolute_max__number_of_maxima_7
A00/A00001,0.0,0.0,1.0,1.0,54.674098,53.922806,0.012433,-0.000029,-2.656584e-07,-0.001062,...,2.014701,2.392839,0.0,5.174527,20.063794,12.127403,12.850602,9.632753,13.735235,0.957992
A00/A00002,0.0,0.0,0.0,1.0,17.956569,19.082151,0.007993,0.000018,-5.434316e-07,-0.005429,...,2.223643,2.640601,0.0,3.121748,16.492801,6.847941,4.524595,3.721042,9.852359,0.829145
A00/A00004,0.0,0.0,0.0,1.0,55.476331,50.275993,0.010847,-0.000048,-1.850963e-06,-0.018121,...,1.673896,1.932486,0.0,4.077444,18.547557,7.253973,5.800808,4.950943,7.281003,0.962595
A00/A00005,0.0,0.0,1.0,1.0,42.030686,57.751486,0.015752,0.000010,1.219631e-05,-0.018051,...,2.198774,2.602241,0.0,4.068822,22.965486,14.629741,14.901379,12.418248,18.788182,0.958226
A00/A00006,0.0,0.0,0.0,1.0,6.403409,407.454707,0.031925,0.000034,-2.132815e-05,-0.034091,...,1.801298,2.108223,0.0,8.113196,29.527131,13.913118,9.173725,8.313102,22.776293,2.483766


In [71]:
# split to the test data
y = y.sort_index()
X_train, X_test, y_train, y_test = train_test_split(extracted_features, y, test_size=0.3, shuffle=True, stratify=y, random_state=SEED)

In [72]:
# select importance features
X_train = select_features(X_train, y_train)
X_train.head()

,"0__agg_autocorrelation__f_agg_""var""__maxlag_40",0__spkt_welch_density__coeff_5,0__percentage_of_reoccurring_values_to_all_values,0__ratio_beyond_r_sigma__r_2.5,"0__fft_coefficient__attr_""abs""__coeff_12","0__fft_coefficient__attr_""abs""__coeff_39",0__permutation_entropy__dimension_7__tau_1,"0__fft_coefficient__attr_""abs""__coeff_38",0__permutation_entropy__dimension_6__tau_1,"0__fft_coefficient__attr_""abs""__coeff_34",...,"0__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","0__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.8","0__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0","0__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4",0__ar_coefficient__coeff_7__k_10,"0__matrix_profile__feature_""mean""__threshold_0.98","0__fft_coefficient__attr_""abs""__coeff_74",0__time_reversal_asymmetry_statistic__lag_3,"0__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.2",0__energy_ratio_by_chunks__num_segments_10__segment_focus_6
A00/A00514,0.047620,0.129522,0.285115,0.0355,47.253982,9.738365,2.197498,4.699168,1.868718,21.698352,...,0.053516,0.000003,-0.000108,0.005853,-0.220421,6.332812,5.752629,0.000029,0.000053,0.023142
A03/A03476,0.071925,0.540827,0.580645,0.0290,6.987139,5.065211,2.408563,7.019762,2.043338,2.546651,...,0.069309,0.000368,0.000568,0.006784,0.074042,5.219208,10.950842,0.000819,0.000059,0.118375
A02/A02562,0.054804,0.259394,0.664430,0.0345,7.708136,22.425957,2.731415,8.611246,2.296545,0.765548,...,0.058827,0.001845,0.000567,0.008473,-0.273136,4.708798,16.237726,0.001426,0.000106,0.145610
A06/A06335,0.105084,0.454827,0.627706,0.0300,9.262161,5.373843,2.441973,9.369700,2.074178,11.014708,...,0.026234,0.000545,0.000165,0.001433,0.088041,10.189959,6.006212,0.000297,0.000006,0.008581
A00/A00232,0.155380,0.835121,0.606618,0.0495,2.189907,10.493366,1.816469,7.381677,1.567110,36.592918,...,0.079673,0.000659,0.000111,0.003124,0.009299,10.380908,13.039653,0.000045,0.000027,0.004423


In [73]:
# select features from test data
X_test = X_test[X_train.columns]
X_test

,"0__agg_autocorrelation__f_agg_""var""__maxlag_40",0__spkt_welch_density__coeff_5,0__percentage_of_reoccurring_values_to_all_values,0__ratio_beyond_r_sigma__r_2.5,"0__fft_coefficient__attr_""abs""__coeff_12","0__fft_coefficient__attr_""abs""__coeff_39",0__permutation_entropy__dimension_7__tau_1,"0__fft_coefficient__attr_""abs""__coeff_38",0__permutation_entropy__dimension_6__tau_1,"0__fft_coefficient__attr_""abs""__coeff_34",...,"0__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","0__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.8","0__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.0","0__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4",0__ar_coefficient__coeff_7__k_10,"0__matrix_profile__feature_""mean""__threshold_0.98","0__fft_coefficient__attr_""abs""__coeff_74",0__time_reversal_asymmetry_statistic__lag_3,"0__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.2",0__energy_ratio_by_chunks__num_segments_10__segment_focus_6
A03/A03063,0.124610,30.519479,0.574751,0.0525,255.589575,412.429575,2.136523,75.508078,1.826495,42.551880,...,-0.199950,-0.000496,-0.001118,0.023055,0.074602,7.218169,89.753829,0.004186,0.001313,0.136417
A04/A04173,0.057243,0.310094,0.622047,0.0360,15.246564,4.068136,2.464100,8.497580,2.088214,11.312946,...,0.047242,0.001533,0.000624,0.002681,-0.661161,7.224211,28.375628,0.001820,0.000015,0.114623
A02/A02969,0.059561,0.324949,0.565625,0.0375,5.556959,4.998362,2.358754,9.495262,2.015475,16.741857,...,0.046654,0.004446,0.000843,0.003012,0.200081,6.576251,4.146741,0.002096,0.000023,0.104391
A05/A05219,0.052276,0.234174,0.740223,0.0275,12.825821,4.321406,2.538606,12.349711,2.133237,29.393995,...,0.050600,-0.000611,0.000011,0.006177,0.309764,4.522190,17.645291,-0.000227,0.000082,0.063921
A02/A02814,0.061262,0.593350,0.649390,0.0350,12.392711,12.183370,2.358732,13.772405,1.992779,8.736287,...,0.051987,0.001402,0.000577,0.004824,-0.259466,4.533793,12.442509,0.000609,0.000071,0.084585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A07/A07910,0.071926,0.271041,0.681234,0.0320,4.694455,44.514580,2.692012,9.799801,2.269634,13.523191,...,0.034773,0.000096,-0.000054,0.003344,0.135841,6.263901,12.033730,0.000169,0.000031,0.113012
A02/A02808,0.129950,0.534146,0.657289,0.0335,6.721959,48.710690,1.791819,17.051263,1.545091,10.142462,...,0.038587,0.000329,0.000135,0.002034,-0.234493,9.124579,8.815121,-0.000019,0.000016,0.006671
A06/A06738,0.056221,0.316210,0.786885,0.0275,4.738597,12.687680,3.618041,2.978803,2.947666,32.785528,...,0.084917,-0.002338,0.000234,0.029793,0.001676,2.733050,16.898543,-0.000322,0.000954,0.143320
A00/A00924,0.061546,0.241005,0.645270,0.0250,14.067329,15.800903,2.920280,11.128941,2.432324,6.084728,...,0.024672,0.000883,0.000108,0.010091,-0.289555,7.054381,1.719787,0.000099,0.000145,0.123032


### CatBoost

### Features - tsfresh statistic

In [59]:
# init, train and predict model
model = CatBoostClassifier(iterations=100, verbose=False, random_seed=SEED)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.4190179763845328

## NN 

In [7]:
# define cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=SEED, stratify=y)

In [9]:
# data normalization and move to device
scaler = StandardScaler()
X_train_scaled = torch.Tensor(scaler.fit_transform(X_train)).to(device)
X_test_scaled = torch.Tensor(scaler.transform(X_test)).to(device)
X_val_scaled = torch.Tensor(scaler.transform(X_val)).to(device)
X_train_scaled

tensor([[ 0.6819,  0.6708,  0.6578,  ...,  0.5387,  0.6629,  0.8921],
        [-0.4022, -0.4997, -0.5464,  ..., -0.3781, -0.4346, -0.4875],
        [-0.0636, -0.0575, -0.0495,  ...,  1.5560,  1.6214,  1.7685],
        ...,
        [ 0.8838,  1.4747,  2.1137,  ..., -0.1629, -0.1865, -0.2271],
        [-0.2084, -0.4565, -0.7106,  ..., -0.1348, -0.7072, -1.6781],
        [-1.0791, -1.0978, -1.1275,  ..., -0.2063, -0.0705,  0.1246]],
       device='cuda:0')

In [10]:
# move to device
y_train = torch.LongTensor(y_train).to(device)
y_test = torch.LongTensor(y_test).to(device)
y_val = torch.LongTensor(y_val).to(device)

In [346]:
# define loaders
torch.manual_seed(SEED)
batch_size = 10
train = torch.utils.data.TensorDataset(X_train_scaled, y_train)
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

test = torch.utils.data.TensorDataset(X_test_scaled, y_test)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)

val = torch.utils.data.TensorDataset(X_val_scaled, y_val)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=True)

## 2. CNN

In [347]:
# define CNN class
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden_1 = nn.Conv1d(1, 40, 6)
        self.hidden_2 = nn.Conv1d(40, 20, 6)
        self.hidden_3 = nn.Conv1d(20, 8, 6)
        self.hidden_4 = nn.Conv1d(8, 4, 6)
        self.hidden_5 = nn.Conv1d(4, 2, 6)
        self.hidden_6 = nn.Linear(106, 50)
        self.output = nn.Linear(50, 4)

        self.pool = nn.MaxPool1d(6, stride=2)
        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        
        x = self.hidden_1(x)
        x = self.pool(x)
        x = self.hidden_2(x)
        x = self.pool(x)
        x = self.hidden_3(x)
        x = self.pool(x)
        x = self.hidden_4(x)
        x = self.pool(x)
        x = self.hidden_5(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = torch.relu(self.hidden_6(x))
        x = self.dropout(x) 
        x = self.output(x)

        return x

In [348]:
# fix seeds
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [349]:
# define model, loss and optimizer
model = CNN().to(device)
model.to(device)
criterion = nn.CrossEntropyLoss()
criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
metric = 0

In [350]:
# train and validate model
epochs = 20
for e in range(epochs):
    model.train()
    for leafs, labels in tqdm(train_loader):
        optimizer.zero_grad()
        output = model(leafs.view(-1, 1, 2000))
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    else:
        model.eval()
        preds = []
        trues = []
        with torch.no_grad():
            for leafs, labels in test_loader:
                output = model(leafs.view(-1, 1, 2000))
                loss = criterion(output, labels)
                
                _, predicted = torch.max(output.data, 1)
                preds.extend(predicted.to('cpu').detach())
                trues.extend(labels.to('cpu').detach())
            
            score = balanced_accuracy_score(trues, preds)
            print(score)
            if score > metric:
                print('Saving model')
                torch.save(model.state_dict(), 'models/model.pt')
                metric = score

100%|██████████| 538/538 [00:05<00:00, 99.13it/s] 


0.26666666666666666
Saving model


100%|██████████| 538/538 [00:04<00:00, 115.87it/s]


0.30009236039586223
Saving model


100%|██████████| 538/538 [00:03<00:00, 150.85it/s]


...

0.4925980754086794


100%|██████████| 538/538 [00:04<00:00, 111.56it/s]


0.440461503440658


100%|██████████| 538/538 [00:05<00:00, 95.96it/s] 


0.5175933597722078
Saving model


## 3.FCNN

In [329]:
# define NN class
class Network(nn.Module):
    def __init__(self):
        super().__init__()

        self.hidden_1 = nn.Linear(2000, 1000)
        self.hidden_2 = nn.Linear(1000, 500)
        self.hidden_3 = nn.Linear(500, 250)
        self.hidden_4 = nn.Linear(250, 50)
        self.output = nn.Linear(50, 4)

        self.dropout = nn.Dropout(0.25)
    
    def forward(self, x):
        
        x = torch.relu(self.hidden_1(x))
        x = torch.relu(self.hidden_2(x))
        x = self.dropout(x)
        x = torch.relu(self.hidden_3(x))
        x = self.dropout(x)
        x = torch.relu(self.hidden_4(x))
        x = self.dropout(x)
        x = self.output(x)

        return x

In [340]:
# fix seeds
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [341]:
# define model, loss and optimizer
model = Network().to(device)
model.to(device)
criterion = nn.CrossEntropyLoss()
criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [342]:
# train and validate model
epochs = 20
model.train()
for e in range(epochs):
    for leafs, labels in tqdm(train_loader):
    
        optimizer.zero_grad()
        output = model(leafs)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    else:
        model.eval()
        preds = []
        trues = []
        with torch.no_grad():
            for leafs, labels in test_loader:
                output = model(leafs)
                loss = criterion(output, labels)
                
                _, predicted = torch.max(output.data, 1)
                preds.extend(predicted.to('cpu').detach())
                trues.extend(labels.to('cpu').detach())
            
            score = balanced_accuracy_score(trues, preds)
            print(score)

100%|██████████| 538/538 [00:01<00:00, 342.29it/s]


0.25


100%|██████████| 538/538 [00:01<00:00, 333.80it/s]


0.25


100%|██████████| 538/538 [00:01<00:00, 412.52it/s]


...

0.2777320246054102


100%|██████████| 538/538 [00:01<00:00, 425.17it/s]


0.28637479143252315


100%|██████████| 538/538 [00:01<00:00, 381.37it/s]


0.2979709902345094


## 4.LSTM

In [336]:
# define RNN class
class RNN(nn.Module):
    
    def __init__(self, input_size, output_size, n_hidden, n_layers):
        super().__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_size, n_hidden, n_layers, batch_first=True)
        self.lstm_2 = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(0.2)
        self.fcl = nn.Linear(n_hidden, output_size)
    
    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = self.drop(out)
        out, hidden = self.lstm_2(out, hidden)
        out = self.drop(out)
        out = out.view(-1, self.n_hidden)
        out = self.fcl(out)
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data

        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                    weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [337]:
# fix seeds
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [338]:
# define model, loss and optimizer
inputs = 2000
outputs = 4
n_hidden = 100
n_layers = 2

model = RNN(inputs, outputs, n_hidden, n_layers).to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().to(device)
model

RNN(
  (lstm): LSTM(2000, 100, num_layers=2, batch_first=True)
  (lstm_2): LSTM(100, 100, num_layers=2, batch_first=True)
  (drop): Dropout(p=0.2, inplace=False)
  (fcl): Linear(in_features=100, out_features=4, bias=True)
)

In [339]:
# train and validate model
epochs = 20
for e in range(epochs):
    model.train()
    h = model.init_hidden(batch_size)
    for leafs, labels in tqdm(train_loader):
        if len(labels) < batch_size:
            continue

        h = tuple((each.data for each in h))

        opt.zero_grad()
        leafs = leafs.view(-1, 1, 2000) 
        output, h = model(leafs, h)
        loss = criterion(output, labels)
        loss.backward()
        opt.step()
        
    else:
        model.eval()
        preds = []
        trues = []
        with torch.no_grad():
            h = model.init_hidden(batch_size)

            for leafs, labels in test_loader:
                if len(labels) < batch_size:
                    continue
                
                h = tuple((each.data for each in h))
                leafs = leafs.view(-1, 1, 2000) 
                output, h = model(leafs, h)
                loss = criterion(output, labels)
                
                _, predicted = torch.max(output.data, 1)
                preds.extend(predicted.to('cpu').detach())
                trues.extend(labels.to('cpu').detach())
            
            score = balanced_accuracy_score(trues, preds)
            print(score)


100%|██████████| 538/538 [00:02<00:00, 209.10it/s]


0.25


100%|██████████| 538/538 [00:02<00:00, 231.45it/s]


0.27366141206594563


100%|██████████| 538/538 [00:02<00:00, 250.55it/s]


...

0.28283255257076306


100%|██████████| 538/538 [00:02<00:00, 200.10it/s]


0.2937110625021329


100%|██████████| 538/538 [00:02<00:00, 203.58it/s]


0.3049553451499965


# The best model is CNN

## Final validation model

In [130]:
# load the best model
model = CNN().to(device)
state_dict = torch.load(os.path.join('models', 'model.pt'), map_location=device)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [131]:
# validate model
model.eval()
preds = []
trues = []
with torch.no_grad():
    for leafs, labels in val_loader:
        output = model(leafs.view(-1, 1, 2000))
        loss = criterion(output, labels)
        
        _, predicted = torch.max(output.data, 1)
        preds.extend(predicted.to('cpu').detach())
        trues.extend(labels.to('cpu').detach())
    
    score = balanced_accuracy_score(trues, preds)
    print(score)

0.4841014580673594
